In [3]:
import json
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage,AIMessage

In [4]:
llm = init_chat_model(
    model="gemini-2.5-flash-lite",
    model_provider="google_genai",
    temperature=0.2,
    api_key=os.getenv("GOOGLE_GEMINI_API_KEY"),
)

In [5]:
DATASET_PROMPT = """
You are generating test cases for evaluating a prompt.

Generate a JSON array of objects.
Each object must have a "task" & a "format" field

The task should ask for ONE of the following outputs:
- Python code
- JSON configuration
- A regular expression

The format field should include the type of task i.e. it is python/json/regex

Do NOT include explanations.
Return ONLY valid JSON.
Generate exactly 3 tasks.
"""

In [6]:
def generate_dataset():
    messages = [HumanMessage(content=DATASET_PROMPT), AIMessage(content="```json")]

    response = llm.invoke(messages, stop=["```"])

    raw_json = response.content.strip()
    return json.loads(raw_json)

In [7]:
def save_dataset(dataset, path="dataset.json"):
    with open(path, "w") as f:
        json.dump(dataset, f, indent=2)

In [8]:
if __name__ == "__main__":
    dataset = generate_dataset()
    print("Generated dataset: ")
    print(dataset)

    save_dataset(dataset, "tasks.json")
    print("Dataset saved.")

Generated dataset: 
[{'task': 'Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.', 'format': 'python'}, {'task': "Generate a JSON configuration for a web server with the following settings: port 8080, enable SSL, and set the root directory to '/var/www/html'.", 'format': 'json'}, {'task': 'Create a regular expression to match email addresses.', 'format': 'regex'}]
Dataset saved.
